In [1]:

import os
import pickle
import sys
import pandas as pd
import numpy as np
import torch

from pathlib import Path
import json
from json import JSONEncoder
import csv


gems_tco_path = "/Users/joonwonlee/Documents/GEMS_TCO-1/src"
sys.path.append(gems_tco_path)
from GEMS_TCO import orderings as _orderings

# Standard libraries
import sys
# Add your custom path
sys.path.append("/cache/home/jl2815/tco")
import os


# Data manipulation and analysis
import pandas as pd
import numpy as np
import pickle
import torch
import torch.optim as optim
import copy                    # clone tensor
import time

# Custom imports
import GEMS_TCO
from GEMS_TCO import kernels
from GEMS_TCO import orbitmap 
from GEMS_TCO import kernels 
from GEMS_TCO import orderings as _orderings 
from GEMS_TCO import load_data_amarel


from typing import Optional, List, Tuple
from pathlib import Path

load estimates for initial parameters

In [2]:
input_path = "C:\\Users\\joonw\\tco\\GEMS_TCO-2\\Exercises\\st_model\\estimates\\"
input_filename = "full_estimation_1250_july24.pkl"
input_filepath = os.path.join(input_path, input_filename)
# Load pickle
with open(input_filepath, 'rb') as pickle_file:
    amarel_map1250= pickle.load(pickle_file)

df_1250 = pd.DataFrame()
for key in amarel_map1250:
    tmp = pd.DataFrame(amarel_map1250[key][0].reshape(1, -1), columns=['sigmasq', 'range_lat', 'range_lon', 'advec_lat', 'advec_lon', 'beta', 'nugget'])
    tmp['loss'] = amarel_map1250[key][1]
    df_1250 = pd.concat((df_1250, tmp), axis=0)

date_range = pd.date_range(start='07-01-24', end='07-31-24')

# Ensure the number of dates matches the number of rows in df_1250
if len(date_range) == len(df_1250):
    df_1250.index = date_range
else:
    print("The number of dates does not match the number of rows in the DataFrame.")

df = df_1250.copy( deep=True )

In [4]:

class load_data:
    def __init__(self, datapath):
        self.datapath = datapath
    
    def load_mm20k_data_bymonthyear(self, lat_lon_resolution= [10,10], mm_cond_number=10, years_=['2024'], months_=[7,8]):

        ## Load the one dictionary to set spaital coordinates
        # filepath = "C:/Users/joonw/TCO/GEMS_data/data_2023/sparse_cen_map23_01.pkl"
        filepath =  Path(self.datapath) / "pickle_2023\\coarse_cen_map23_01.pkl"
        
        with open(filepath, 'rb') as pickle_file:
            coarse_dict_24_1 = pickle.load(pickle_file)

        sample_df = coarse_dict_24_1['y23m01day01_hm02:12']
        sample_key = coarse_dict_24_1.get('y23m01day01_hm02:12')
        if sample_key is None:
            print("Key 'y23m01day01_hm02:12' not found in the dictionary.")

        rho_lat = lat_lon_resolution[0]          
        rho_lon = lat_lon_resolution[1]
        lat_n = sample_df['Latitude'].unique()[::rho_lat]
        lon_n = sample_df['Longitude'].unique()[::rho_lon]

        # Set spatial coordinates for each dataset
        coarse_dicts = {}
        years = years_
        for year in years:
            for month in range(months_[0], months_[1]):  # Iterate over all months
                # filepath = f"C:/Users/joonw/TCO/GEMS_data/data_{year}/sparse_cen_map{year[2:]}_{month:02d}.pkl"
                filepath = Path(self.datapath) / f"pickle_{year}/coarse_cen_map{year[2:]}_{month:02d}.pkl"
                with open(filepath, 'rb') as pickle_file:
                    loaded_map = pickle.load(pickle_file)
                    for key in loaded_map:
                        tmp_df = loaded_map[key]
                        coarse_filter = (tmp_df['Latitude'].isin(lat_n)) & (tmp_df['Longitude'].isin(lon_n))
                        coarse_dicts[f"{year}_{month:02d}_{key}"] = tmp_df[coarse_filter].reset_index(drop=True)

        key_idx = sorted(coarse_dicts)
        if not key_idx:
            raise ValueError("coarse_dicts is empty")

        # extract first hour data because all data shares the same spatial grid
        data_for_coord = coarse_dicts[key_idx[0]]
        x1 = data_for_coord['Longitude'].values
        y1 = data_for_coord['Latitude'].values 
        coords1 = np.stack((x1, y1), axis=-1)

        # instance = orbitmap.MakeOrbitdata(data_for_coord, lat_s=5, lat_e=10, lon_s=110, lon_e=120)
        # s_dist = cdist(coords1, coords1, 'euclidean')
        # ord_mm, _ = instance.maxmin_naive(s_dist, 0)

        ord_mm = _orderings.maxmin_cpp(coords1)
        data_for_coord = data_for_coord.iloc[ord_mm].reset_index(drop=True)
        coords1_reordered = np.stack((data_for_coord['Longitude'].values, data_for_coord['Latitude'].values), axis=-1)
        # nns_map = instance.find_nns_naive(locs=coords1_reordered, dist_fun='euclidean', max_nn=mm_cond_number)
        nns_map=_orderings.find_nns_l2(locs= coords1_reordered  ,max_nn = mm_cond_number)
        return coarse_dicts, ord_mm, nns_map

    def load_working_data_byday(self, coarse_dicts,  ord_mm, nns_map, idx_for_datamap=[0,8]):
        key_idx = sorted(coarse_dicts)
        if not key_idx:
            raise ValueError("coarse_dicts is empty")
        analysis_data_map = {}
        for i in range(idx_for_datamap[0],idx_for_datamap[1]):
            tmp = coarse_dicts[key_idx[i]].copy()
            tmp['Hours_elapsed'] = np.round(tmp['Hours_elapsed']-477700)
            tmp = tmp.iloc[ord_mm, :4].to_numpy()  # reorder the data
            tmp = torch.from_numpy(tmp).double()
            analysis_data_map[key_idx[i]] = tmp

        aggregated_data = pd.DataFrame()
        for i in range(idx_for_datamap[0],idx_for_datamap[1]):
            tmp = coarse_dicts[key_idx[i]].copy()
            tmp['Hours_elapsed'] = np.round(tmp['Hours_elapsed']-477700)
            tmp = tmp.iloc[ord_mm].reset_index(drop=True)  
            aggregated_data = pd.concat((aggregated_data, tmp), axis=0)

        aggregated_data = aggregated_data.iloc[:, :4].to_numpy()
        #aggregated_data = torch.from_numpy(aggregated_data).float() 
        aggregated_data = torch.from_numpy(aggregated_data).double()

        return analysis_data_map, aggregated_data


    def reorder_data(self, analysis_data_map, aggregated_data, key_order):
        # key_order = [0, 1, 2, 4, 3, 5, 7, 6]
        keys = list(analysis_data_map.keys())
        reordered_dict = {keys[key]: analysis_data_map[keys[key]] for key in key_order}
        reorder_keys = list(reordered_dict.keys())
        data_frames = []

        for key in reorder_keys:
            tensor_data = reordered_dict[key]
            if isinstance(tensor_data, torch.Tensor):
                tensor_data = tensor_data.numpy()  # Convert tensor to NumPy array
                tensor_df = pd.DataFrame(tensor_data)  # Convert NumPy array to DataFrame
            else:
                tensor_df = tensor_data  # If it's already a DataFrame
            data_frames.append(tensor_df)

        reordered_df = pd.concat(data_frames, axis=0)
        reordered_df = reordered_df.to_numpy()
        reordered_df = torch.from_numpy(reordered_df).double()

        return reordered_dict, reordered_df



    def load_working_data_by_quarterday(self, coarse_dicts, ord_mm, nns_map, which_group, qrt_idx, avg_days):
        keys = sorted(coarse_dicts)
        if not keys:
            raise ValueError("coarse_dicts is empty")
        
        avg_idx = 8 * avg_days
        analysis_data_map = {}
        entire_data = []

        # qrt_idx takes 1, 2, 3, 4 for 4 quarters
        for i in range(which_group-1, which_group ):
            idx_quarter = [[avg_idx * i + 8 * j + 2 * (qrt_idx - 1), avg_idx * i + 8 * j + (2 * qrt_idx - 1)] for j in range(avg_days)]
            idx_quarter = [item for sublist in idx_quarter for item in sublist]

            aggregated_data = []
            for key_idx in idx_quarter:
                tmp = coarse_dicts[keys[key_idx]].copy()
                tmp['Hours_elapsed'] = np.round(tmp['Hours_elapsed'] - 477700)
                tmp['new_key'] = key_idx % 8
                aggregated_data.append(tmp)
                
                tmp = tmp.iloc[ord_mm, [0,1,2,3,5]  ].to_numpy()
                tmp = torch.from_numpy(tmp).double()

                analysis_data_map[f'unit_{i}_quarter_{key_idx % 8}'] = tmp

            aggregated_data = pd.concat(aggregated_data, axis=0)
            aggregated_data = aggregated_data[['Latitude', 'Longitude', 'ColumnAmountO3', 'new_key']].groupby(['Latitude', 'Longitude', 'new_key']).mean().reset_index()
            aggregated_data['quarter'] = qrt_idx
        
            aggregated_data = aggregated_data.iloc[:, :5 ].to_numpy()
            aggregated_data = torch.from_numpy(aggregated_data).double()
            entire_data.append(aggregated_data)

        entire_data = torch.cat(entire_data, dim=0)
        entire_data = entire_data[:, [0, 1, 3, 2]]
        return analysis_data_map, entire_data



In [10]:
lat_lon_resolution = [20,20]
years = ['2024']
month_range =[7,8]
v= 0.5
lr = 0.02
days=10
mm_cond_number=10
epochs= 100
nheads = 10

result_q1 = {}
result_q2 = {}
result_q3 = {}
result_q4 = {}


avg_days = 4

datapath = "C:\\Users\\joonw\\tco\\Extracted_data\\"
instance = load_data(datapath)
mm_cond_number = 10
coarse_dicts, ord_mm, nns_map = instance.load_mm20k_data_bymonthyear( lat_lon_resolution= lat_lon_resolution, mm_cond_number=mm_cond_number,years_=years, months_=month_range)

# (31//avg_days)+1

for group_idx in range(1, 8):
    print(f'\n Group {group_idx} data size per day: { (200/lat_lon_resolution[0])*(100/lat_lon_resolution[0])  } \n')

    # parameters

    params = list(df.iloc[ 4*(group_idx-1)][:-1])
    params = torch.tensor(params, dtype=torch.float64, requires_grad=True)
    analysis_data_map_q1, entire_data_q1 = instance.load_working_data_by_quarterday( coarse_dicts, ord_mm, nns_map, which_group=group_idx, qrt_idx=1, avg_days= avg_days)


entire_data_q1


 Group 1 data size per day: 50.0 


 Group 2 data size per day: 50.0 


 Group 3 data size per day: 50.0 


 Group 4 data size per day: 50.0 


 Group 5 data size per day: 50.0 


 Group 6 data size per day: 50.0 


 Group 7 data size per day: 50.0 



tensor([[  5.0250, 110.0250, 273.2014,   0.0000],
        [  5.0250, 110.0250, 270.8236,   1.0000],
        [  5.0250, 111.0250, 272.7497,   0.0000],
        [  5.0250, 111.0250, 271.7945,   1.0000],
        [  5.0250, 112.0250, 272.6109,   0.0000],
        [  5.0250, 112.0250, 271.2740,   1.0000],
        [  5.0250, 113.0250, 274.1084,   0.0000],
        [  5.0250, 113.0250, 271.1791,   1.0000],
        [  5.0250, 114.0250, 272.0381,   0.0000],
        [  5.0250, 114.0250, 270.5241,   1.0000],
        [  5.0250, 115.0250, 271.2672,   0.0000],
        [  5.0250, 115.0250, 270.2809,   1.0000],
        [  5.0250, 116.0250, 270.5139,   0.0000],
        [  5.0250, 116.0250, 269.6774,   1.0000],
        [  5.0250, 117.0250, 271.3630,   0.0000],
        [  5.0250, 117.0250, 271.5377,   1.0000],
        [  5.0250, 118.0250, 270.5465,   0.0000],
        [  5.0250, 118.0250, 277.6002,   1.0000],
        [  5.0250, 119.0250, 269.5345,   0.0000],
        [  5.0250, 119.0250, 268.9179,   1.0000],


In [7]:
lat_lon_resolution = [20,20]
years = ['2024']
month_range =[7,8]
v= 0.5
lr = 0.02
days=10
mm_cond_number=10
epochs= 100
nheads = 10

result_q1 = {}
result_q2 = {}
result_q3 = {}
result_q4 = {}


avg_days = 4

datapath = "C:\\Users\\joonw\\tco\\Extracted_data\\"
instance = load_data(datapath)
mm_cond_number = 10
coarse_dicts, ord_mm, nns_map = instance.load_mm20k_data_bymonthyear( lat_lon_resolution= lat_lon_resolution, mm_cond_number=mm_cond_number,years_=years, months_=month_range)

# (31//avg_days)+1

for group_idx in range(1, 8):
    print(f'\n Group {group_idx} data size per day: { (200/lat_lon_resolution[0])*(100/lat_lon_resolution[0])  } \n')

    # parameters

    params = list(df.iloc[ 4*(group_idx-1)][:-1])
    params = torch.tensor(params, dtype=torch.float64, requires_grad=True)
    analysis_data_map_q1, entire_data_q1 = instance.load_working_data_by_quarterday( coarse_dicts, ord_mm, nns_map, which_group=group_idx, qrt_idx=1, avg_days= avg_days)
    
    params = list(df.iloc[ 4*(group_idx-1)][:-1])
    params = torch.tensor(params, dtype=torch.float64, requires_grad=True)
    analysis_data_map_q2, entire_data_q2 = instance.load_working_data_by_quarterday( coarse_dicts, ord_mm, nns_map, which_group=group_idx, qrt_idx=2, avg_days= avg_days)
    
    params = list(df.iloc[ 4*(group_idx-1)][:-1])
    params = torch.tensor(params, dtype=torch.float64, requires_grad=True)
    analysis_data_map_q3, entire_data_q3 = instance.load_working_data_by_quarterday( coarse_dicts, ord_mm, nns_map, which_group=group_idx, qrt_idx=3, avg_days= avg_days)
    
    params = list(df.iloc[ 4*(group_idx-1)][:-1])
    params = torch.tensor(params, dtype=torch.float64, requires_grad=True)
    analysis_data_map_q4, entire_data_q4 = instance.load_working_data_by_quarterday( coarse_dicts, ord_mm, nns_map, which_group=group_idx, qrt_idx=4, avg_days= avg_days)
    
    print(f'group_idx {group_idx+1}, data size per hour: {entire_data_q1.shape[0]}, smooth: {v}')
    print(lat_lon_resolution, mm_cond_number , params, v,lr)
    
    params = list(df_1250.iloc[group_idx][:-1])
    params = torch.tensor(params, dtype=torch.float64, requires_grad=True)

    model_q1 = kernels.model_fitting(
            smooth=v,
            input_map= analysis_data_map_q1,
            aggregated_data= entire_data_q1,
            nns_map=nns_map,
            mm_cond_number=mm_cond_number,
            nheads = nheads
        )

    model_q2 = kernels.model_fitting(
            smooth=v,
            input_map=analysis_data_map_q2,
            aggregated_data=entire_data_q2,
            nns_map=nns_map,
            mm_cond_number=mm_cond_number,
            nheads = nheads
        )

    model_q3 = kernels.model_fitting(
            smooth=v,
            input_map=analysis_data_map_q3,
            aggregated_data=entire_data_q3,
            nns_map=nns_map,
            mm_cond_number=mm_cond_number,
            nheads = nheads
        )

    model_q4 = kernels.model_fitting(
            smooth=v,
            input_map=analysis_data_map_q4,
            aggregated_data=entire_data_q4,
            nns_map=nns_map,
            mm_cond_number=mm_cond_number,
            nheads = nheads
        )
    

    start_time = time.time()
    # optimizer = optim.Adam([params], lr=0.01)  # For Adam
    optimizer, scheduler = model_q1.optimizer_fun(params, lr=0.01, betas=(0.9, 0.8), eps=1e-8, step_size=20, gamma=0.9)    
    
    out_q1, epoch = model_q1.run_full(params, optimizer,scheduler, model_q1.matern_cov_anisotropy_v05, epochs=epochs)
    out_q2 = model_q2.run_full(params, optimizer,scheduler, model_q2.matern_cov_anisotropy_v05, epochs=epochs)
    out_q3 = model_q3.run_full(params, optimizer,scheduler, model_q3.matern_cov_anisotropy_v05, epochs=epochs)
    out_q4 = model_q4.run_full(params, optimizer,scheduler, model_q4.matern_cov_anisotropy_v05, epochs=epochs)
    result_q1[group_idx+1] = out_q1
    result_q2[group_idx+1] = out_q2
    result_q3[group_idx+1] = out_q3
    result_q4[group_idx+1] = out_q4

    end_time = time.time()
    epoch_time = end_time - start_time
    print(f'group_idx {group_idx + 1} took {epoch_time/2:.2f} for each morning and noon')






 Group 1 data size per day: 50.0 

group_idx 2, data size per hour: 100, smooth: 0.5
[20, 20] 10 tensor([ 2.4793e+01,  1.5845e+00,  1.7182e+00,  9.0885e-03, -1.0730e-01,
         1.3104e-01,  2.7172e+00], dtype=torch.float64, requires_grad=True) 0.5 0.02
FINAL STATE: Epoch 100, Loss: 115.17850850988697, 
 vecc Parameters: [ 2.35885897e+01  2.83081303e+00  2.77694861e+00 -2.18183046e-02
 -7.11644834e-02  8.20515711e-03  6.78789892e-01]
FINAL STATE: Epoch 100, Loss: 114.93692153521754, 
 vecc Parameters: [ 2.31086570e+01  3.32602781e+00  3.26188336e+00  1.11338617e-01
 -2.08904947e-01 -7.96766784e-04  2.15580617e-01]
FINAL STATE: Epoch 100, Loss: 123.95686058163915, 
 vecc Parameters: [ 2.28259748e+01  3.61269298e+00  3.51942594e+00  3.77234410e-02
  9.68388859e-03 -9.81099548e-02  9.53452767e-02]
FINAL STATE: Epoch 100, Loss: 128.28839378969286, 
 vecc Parameters: [22.75633693  3.79319402  3.70338232  0.17804059  0.19207337 -0.06625913
 -0.08508345]
group_idx 2 took 1.11 for each morni